In [2]:
#%pip install pandas-datareader
%pip install yfinance
#!pip install yfinance --upgrade --no-cache-dir

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

In [4]:
pd.set_option('display.float_format', lambda x: '%.8f' % x) 
# change the visualization of a float to eight decimal digits.

In [5]:
binance = pd.read_csv('data/final/saldo_binance.csv', index_col=0)
trezor = pd.read_csv('data/final/saldo_trezor.csv', index_col=0)
binance_complete = pd.read_csv('data/final/binance_dates.csv', index_col=0)

In [6]:
tickers = list(set(binance.index).union(trezor.index).union(binance_complete.index))
tickers = ['ETH', 'MATIC', 'BTC']

In [8]:
# Set the start and end dates for the historical data
start_date = (datetime.now()- timedelta(days=1)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

# Fetch the historical data for each ticker using Yahoo Finance API
hist_data = {}
for ticker in tickers:
    data = yf.download(ticker + '-USD', start=start_date, end=end_date)
    hist_data[ticker] = data['Close']

# Combine the historical data for all tickers into a single dataframe
df = pd.concat(hist_data, axis=1, keys=tickers)

# Print the head of the combined dataframe
print(df.head())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                     ETH      MATIC            BTC
Date                                              
2023-03-20 1735.32128906 1.10409701 27767.23632812


In [9]:
df

,ETH,MATIC,BTC
Date,,,
2023-03-20,1735.32128906,1.10409701,27767.23632812


In [13]:
data = pd.DataFrame(index=tickers)
data['Amount'] = 0

data = trezor.copy()
tickers = ['BTC', 'ETH']

In [15]:
data.loc['BTC','Amount'] += binance.loc['BTC','Balance']
data.loc['BTC','BRL_spent'] += binance.loc['BTC','BRL_spent']

In [16]:
data

,Amount,BRL_spent,Mean_price
BRL,15091.74000000,15091.74000000,1.00000000
ETH,0.52902128,4820.77315853,9112.62616605
BTC,0.14965955,17217.83666190,114919.63277309


In [17]:
df

,ETH,MATIC,BTC
Date,,,
2023-03-20,1735.32128906,1.10409701,27767.23632812


In [11]:
for token in tickers:
    print(token)
    data.loc[token,'Amount'] += trezor.loc[token,'Amount'] 
    data.loc[token,'Amount'] += binance.loc[token,'Balance']

ETH


KeyError: 'ETH'

In [ ]:
binance

,UTC_Time,Balance,BRL_spent,Mean_price
Coin,,,,
BTC,2022-01-04 19:50:53,0.002010,250.000000,124380.584688
BUSD,2022-02-19 02:28:25,0.006698,0.000000,0.000000
LUNA,2022-03-07 17:16:16,0.039680,0.000000,0.000000
MATIC,2022-08-24 14:43:07,116.414976,650.341787,5.586410
ETHW,2022-09-20 01:18:13,0.339235,0.000000,0.000000


In [ ]:
data

,Amount
ETH,1.058043
MATIC,0.000000
BTC,0.000000
